# Analysis of Beam Simulator Images and Brighter-fatter Correction
<br>Owner(s): **Andrew Bradshaw** ([@andrewkbradshaw](https://github.com/LSSTScienceCollaborations/StackClub/issues/new?body=@andrewkbradshaw))
<br>Last Verified to Run: **2018-08-10**
<br>Verified Stack Release: **16.0 and 16.0+22 (w_2018_31)**

This notebook demonstrates the [brighter-fatter systematic error](https://arxiv.org/abs/1402.0725) on images of stars and galaxies illuminated on an ITL-3800C-002 CCD at the [UC Davis LSST beam simulator laboratory](https://arxiv.org/abs/1411.5667). Using a series of images at increasing exposure times, we demonstrate the broadening of image profiles on DM stack shape measurements, and a [possible correction method](https://arxiv.org/abs/1711.06273) which iteratively applies a kernel to restore electrons to the pixels which they were deflected from. To keep things simple, for now we skip DM stack instrument signature removal (ISR) and work on a subset of images which are arrays (500x500) of electron counts in pixels.

### Learning Objectives:

After working through this tutorial you should be able to: 
1. Characterize and measure objects (stars/galaxies) in LSST beam simulator images
2. Test the Brighter-Fatter kernel correction method on those images
3. Build your own tests of stack instrument signature removal algorithms

### Logistics
This notebook is intended to be runnable on `lsst-lspdev.ncsa.illinois.edu` from a local git clone of https://github.com/LSSTScienceCollaborations/StackClub.

## Set-up

In [ ]:
# What version of the Stack am I using?
! echo $HOSTNAME
! eups list -s | grep lsst_distrib

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from astropy.io import fits
import time,glob

# if running stack v16.0, silence a long matplotlib Agg warning with:
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

%matplotlib inline

## Read in an image, then set the variance plane based upon it
Cut-outs of beam simulator star/galaxy images have been placed in the shared data directory at `/project/shared/data/beamsim/bfcorr/`. We skip (for now) most of the instrument signature removal (ISR) steps because these are preprocessed images (bias subtracted, gain corrected). We instead start by reading in one of those `.fits` files and making an image plane `afwImage.ExposureF` as well as a variance plane, which is then ready for characterization and calibration in the following cells.

In [ ]:
import lsst.afw.image as afwImage
from lsst.ip.isr.isrFunctions import updateVariance

# where the data lives, choosing one image to start
fitsglob='/project/shared/data/beamsim/bfcorr/*part.fits'
fitsfilename = np.sort(glob.glob(fitsglob))[19]  

# Read in a single image to an afwImage.ImageF object
image_array=afwImage.ImageF.readFits(fitsfilename)
image = afwImage.ImageF(image_array)
exposure = afwImage.ExposureF(image.getBBox())
exposure.setImage(image)
hdr=fits.getheader(fitsfilename) # the header has some useful info in it
print("Read in ",fitsfilename.split('/')[-1])

# Set the variance plane using the image plane via updateVariance function
gain = 1.0 # because these images are already gain corrected
readNoise = 10.0  # in electrons
updateVariance(exposure.getMaskedImage(), gain, readNoise)

# Another way of setting variance and/or masks?
#mask = afwImage.makeMaskFromArray(np.zeros((4000,4072)).astype('int32'))
#variance = afwImage.makeImageFromArray((readNoise**2 + image_array.array())
#masked_image = afwImage.MaskedImageF(image, mask, variance)
#exposure = afwImage.ExposureF(masked_image)

In [ ]:
# Visualize the image and its electron distribution
plt.figure(figsize=(12,5)),plt.subplots_adjust(wspace=.3)
plt.suptitle('Star/galaxy beam sim segment of '+hdr['CCD_SERN']+'\n '+fitsfilename.split('/')[-1])

plt.subplot(121)
plt.imshow(exposure.getImage().array,vmax=1e3,origin='lower')
plt.colorbar(label='electrons')

plt.subplot(122)
plt.hist(exposure.getImage().array.flatten(),bins=1000,histtype='step')
plt.yscale('log')#,plt.xscale('log')
plt.xlabel('Number of electrons in pixel'),plt.ylabel('Number of pixels')

In [ ]:
# +TODO perhaps some other image stats from 
# https://github.com/lsst/pipe_tasks/blob/master/python/lsst/pipe/tasks/exampleStatsTasks.py

## Perform image characterization

In [ ]:
from lsst.pipe.tasks.characterizeImage import CharacterizeImageTask, CharacterizeImageConfig

# first set a few configs that are specific to the data
charConfig = CharacterizeImageConfig()
#this set the fwhm of the simple PSF to that of the fwhm used in the simulation
charConfig.installSimplePsf.fwhm = .2
charConfig.doMeasurePsf = False
charConfig.doApCorr = False
charConfig.repair.doCosmicRay = False  
# we do have some cosmic rays, but we also subpixel features and an undersampled PSF
charConfig.detection.background.binSize = 10 
#charConfig.background.binSize = 50
charConfig.detection.minPixels = 5
charTask = CharacterizeImageTask(config=charConfig)

# charTask.run?  # works for v16.0+22
charTask.characterize?

In [ ]:
tstart=time.time()
charResult = charTask.characterize(exposure) # charTask.run(exposure) stack v16.0+22
print("Characterization took ",str(time.time()-tstart)[:4]," seconds")
print("Detected ",len(charResult.sourceCat)," objects ")

plt.title('X/Y locations of detections')
plt.plot(charResult.sourceCat['base_SdssCentroid_x'],charResult.sourceCat['base_SdssCentroid_y'],'r.')

In [ ]:
# Looking at the mask plane, which started off as all zeros
# and now has some values of 2^5
maskfoo=exposure.getMask()
print("Unique mask plane values: ",np.unique(maskfoo.array))
print("Mask dictionary entries: ",maskfoo.getMaskPlaneDict())

plt.figure(figsize=(12,5)),plt.subplots_adjust(wspace=.3)
plt.subplot(121)
plt.imshow(maskfoo.array,origin='lower'),plt.colorbar()
plt.subplot(122)
plt.hist(maskfoo.array.flatten()),plt.xlabel('Mask plane values')
plt.yscale('log')

## Perform further image calibration and measurement

In [ ]:
# no need to do astrometry or photometry calibration
#since this is a lab image
from lsst.pipe.tasks.calibrate import CalibrateTask, CalibrateConfig
calConfig = CalibrateConfig()
calConfig.doAstrometry = False
calConfig.doPhotoCal = False
calConfig.detection.minPixels = 15
calConfig.doApCorr = False
calConfig.doDeblend = False   # these are well-separated objects, deblending adds time & trouble
calConfig.detection.background.binSize = 50
calTask = CalibrateTask(config= calConfig, icSourceSchema=charResult.sourceCat.schema)

#calTask.run? # for stack v16.0+22 
calTask.calibrate?

In [ ]:
tstart=time.time()
# for stack v16.0+22, change to calTask.run(charResult.exposure)
calResult = calTask.calibrate(charResult.exposure, background=charResult.background,
                              icSourceCat = charResult.sourceCat)

print("Calibration took ",str(time.time()-tstart)[:4]," seconds")
print("Detected ",len(calResult.sourceCat)," objects ")

In [ ]:
# Looking at the source catalog which has now been attached to the 
src=calResult.sourceCat  #.copy(deep=True) ?
#print(src.asAstropy)

src.writeFits(fitsfilename+'.cat')
# read back in and access via:
#catalog=fits.open(fitsfilename+'.cat')
#catalog[1].data['base_SdssShape_xx'] etc.

plt.figure()
par_names=['base_SdssShape_xx','base_SdssShape_yy','base_SdssShape_flux']
par_mins=[0,0,0]
par_maxs=[5,5,1e6]
n_par=len(par_names)


plt.figure(figsize=(5*n_par,6)),plt.subplots_adjust(wspace=.25)
for par_name,par_min,par_max,i in zip(par_names,par_mins,par_maxs,range(n_par)):
    plt.subplot(2,n_par,i+1)
    plt.scatter(src['base_SdssCentroid_x'],src['base_SdssCentroid_y'],c=src[par_name],marker='o',vmin=par_min,vmax=par_max)
    plt.xlabel('X'),plt.ylabel('Y'),plt.colorbar(label=par_name)


    plt.subplot(2,n_par,n_par+i+1)
    plt.hist(src[par_name],range=[par_min,par_max],bins=20,histtype='step')
    plt.xlabel(par_name)

### Display the image with Firefly and overlay detected objects

In [ ]:
import lsst.afw.display as afwDisplay

# Firefly client imports
from firefly_client import FireflyClient

# Standard libraries in support of Firefly display
from urllib.parse import urlparse, urlunparse, ParseResult
from IPython.display import IFrame, display, Markdown
import os

# Own cell?
my_channel = '{}_test_channel'.format(os.environ['USER'])
server = 'https://lsst-lspdev.ncsa.illinois.edu'

# This needs its own cell
ff='{}/firefly/slate.html?__wsch={}'.format(server, my_channel)
IFrame(ff,1000,600)



In [ ]:
# set the backend and attach to the waiting display channel
afwDisplay.setDefaultBackend('firefly')
afw_display = afwDisplay.getDisplay(frame=1, 
                                    name=my_channel)

# Open the exposure (Firefly knows about mask planes)
afw_display.mtv(exposure)

#Now we’ll overplot sources from the src table onto the image display using the Display’s dot method for plotting markers. 
#Display.dot plots markers individually, so you’ll need to iterate over rows in the SourceTable. 
#Next we display the first 100 sources. We limit the number of sources since plotting the whole catalog 
#is a serial process and takes some time. Because of this, it is more efficient to send a batch of updates to the display, 
#so we enclose the loop in a display.Buffering context, like this:

afw_display.erase()

with afw_display.Buffering():
    for record in src[:]:
        afw_display.dot('o', record.getX(), record.getY(), size=20, ctype='orange')

## Apply the brighter-fatter correction

In [ ]:
from lsst.ip.isr.isrTask import IsrTask # brighterFatterCorrection lives here
isr=IsrTask()

pre_bfcorr_exposure=exposure.clone() #save a copy of the pre-bf corrected image

isr.brighterFatterCorrection?

In [ ]:
# Read in the kernel (determined from e.g. simulations or flat fields)
kernel=fits.getdata('/project/shared/data/beamsim/bfcorr/BF_kernel-ITL_3800C_002.fits')

# Perform the correction
tstart=time.time()
exposure=pre_bfcorr_exposure.clone()
isr.brighterFatterCorrection(exposure,kernel,20,10,False)
print("Brighter-fatter correction took",time.time()-tstart," seconds") #takes 99 seconds for 4kx4k exposure, 21x21 kernel, 20 iterations, 10 thresh

# Plot kernel and image differences
plt.figure(),plt.title('BF kernel')
plt.imshow(kernel),plt.colorbar()

imagediff=(pre_bfcorr_exposure.getImage().array-exposure.getImage().array)

plt.figure(figsize=(16,10))
plt.subplot(231),plt.title('Before')
plt.imshow(pre_bfcorr_exposure.getImage().array,vmin=0,vmax=1e3,origin='lower'),plt.colorbar()
plt.subplot(232),plt.title('After')
plt.imshow(exposure.getImage().array,vmin=0,vmax=1e3,origin='lower'),plt.colorbar()
plt.subplot(233),plt.title('Before - After')
vmin,vmax=-50,50
plt.imshow(imagediff,vmin=vmin,vmax=vmax,origin='lower'),plt.colorbar()

nbins=1000
plt.subplot(234)
plt.hist(pre_bfcorr_exposure.getImage().array.flatten(),bins=nbins,histtype='step',label='before'),plt.yscale('log')
plt.subplot(235)
plt.hist(exposure.getImage().array.flatten(),bins=nbins,histtype='step',label='after'),plt.yscale('log')
plt.subplot(236)
plt.hist(imagediff.flatten(),bins=nbins,histtype='step',label='difference'),plt.yscale('log')
plt.legend()
plt.xlabel('Pixel values [e-]')


### Run the characterization & measurement over the 20 exposures of increasing brightness

In [ ]:
# This cell runs through all of the image parts, and should take around 1 second per image (20 sec total)

do_bf_corr=False  # True or False, makes catalogs (.cat) for all of the corrected and uncorrected images
fitsglob='/project/shared/data/beamsim/bfcorr/*part.fits'
for fitsfilename in np.sort(glob.glob(fitsglob)):
    image_array=afwImage.ImageF.readFits(fitsfilename)
    image = afwImage.ImageF(image_array)

    exposure = afwImage.ExposureF(image.getBBox())
    exposure.setImage(image)

    updateVariance(exposure.getMaskedImage(), gain, readNoise)
    
    # start the characterization and measurement, optionally beginning with the brighter-fatter correction
    tstart=time.time()
    if do_bf_corr:
        isr.brighterFatterCorrection(exposure,kernel,20,10,False)
        #print("Brighter-fatter correction took",str(time.time()-tstart)[:4]," seconds")
    # for stack v16.0+22 use charTask.run() and calTask.run()
    charResult = charTask.characterize(exposure)  
    calResult = calTask.calibrate(charResult.exposure, background=charResult.background,
                                  icSourceCat = charResult.sourceCat)
    src=calResult.sourceCat  #.copy(deep=True) ?
    
    # write out the source catalog
    catfilename=fitsfilename.replace('.fits','.cat')
    if do_bf_corr: catfilename=catfilename.replace('.cat','-bfcorr.cat')
    src.writeFits(catfilename)
    
    print(fitsfilename.split('/')[-1]," char. & calib. took ",str(time.time()-tstart)[:4]," seconds to measure ",len(calResult.sourceCat)," objects ")



In [ ]:
# display some of the source catalog shape measurements
for name in src.schema.getOrderedNames():
    if 'shape' in name.lower():
        print(name)

In [ ]:
# Read in the catalogs, both corrected and uncorrected (this could be improved with pandas)
cat_arr = []
catglob='/project/shared/data/beamsim/bfcorr/ITL*part.cat' # uncorrected catalogs
for catfilename in np.sort(glob.glob(catglob)): cat_arr.append(fits.getdata(catfilename))

bf_cat_arr = []
catglob='/project/shared/data/beamsim/bfcorr/ITL*part-bfcorr.cat' # corrected catalogs
for catfilename in np.sort(glob.glob(catglob)): bf_cat_arr.append(fits.getdata(catfilename))
ncats=len(cat_arr)

In [ ]:
# Show possible issues with source matching which we will remedy with simple matching in the next cell
for i in range(ncats):
    xfoo,yfoo=cat_arr[i]['base_SdssCentroid_x'],cat_arr[i]['base_SdssCentroid_y']
    plt.plot(xfoo,yfoo,'o')
plt.title('Centroids of sequential exposures')

In [ ]:
# Using a fiducial frame as reference, we simply match the catalogs by looking for single objects
# within a max distance

fidframe=10
maxdist=.5

x0s,y0s=cat_arr[fidframe]['base_SdssCentroid_x'],cat_arr[fidframe]['base_SdssCentroid_y']
nspots=len(x0s)
bf_dat=np.empty((ncats,nspots,6)) #x,y,xx,yy,bfxx,bfyy
bf_dat[:]=np.nan

for i in range(ncats):
    # get the centroids of objects in the bf-corrected and uncorrected images
    x1,y1=cat_arr[i]['base_SdssCentroid_x'],cat_arr[i]['base_SdssCentroid_y']
    x1_bf,y1_bf=bf_cat_arr[i]['base_SdssCentroid_x'],bf_cat_arr[i]['base_SdssCentroid_y']
    for j in range(nspots):   # loop over fiducial frame centroids to find matches
        x0,y0=x0s[j],y0s[j]
        # find the matches between the fiducial centroid (x0,y0) and the corrected/uncorrected ones
        bf_gd=np.where(np.sqrt((x1_bf-x0)**2+(y1_bf-y0)**2)<maxdist)[0]
        gd=np.where(np.sqrt((x1-x0)**2+(y1-y0)**2)<maxdist)[0]
        if (len(bf_gd)==1 & len(gd)==1):  # only take single matches
            xx,yy=cat_arr[i]['base_SdssShape_xx'][gd],cat_arr[i]['base_SdssShape_yy'][gd]
            xx_bf,yy_bf=bf_cat_arr[i]['base_SdssShape_xx'][bf_gd],bf_cat_arr[i]['base_SdssShape_yy'][bf_gd]
            bf_dat[i,j,:]=x0,y0,xx,yy,xx_bf,yy_bf  # keep those above measurements

In [ ]:
# Plot the brighter-fatter effect on those shape measurements and the corrected version
# These are good indices to look with defaults: [0,1,6,12,23,35,44,46,52,56,59,69,71,73,90]

# +TODO - change the exposure number to some brightness measurement
# +TODO - get postage stamps in a stackly manner

nfoo=44
plt.figure(figsize=(14,4)),plt.subplots_adjust(wspace=.3)
# grab a postage stamp, +TODO in a stackly manner
sz=11
xc,yc=bf_dat[10,nfoo,0].astype('int')+1,bf_dat[10,nfoo,1].astype('int')+1
stamp=exposure.getImage().array[yc-sz:yc+sz,xc-sz:xc+sz]
plt.subplot(131)
plt.imshow(stamp,origin='lower',norm=LogNorm(1,stamp.max())),plt.colorbar()

plt.subplot(132)
plt.plot(bf_dat[:,nfoo,2],'r.',label='Uncorrected')
plt.plot(bf_dat[:,nfoo,4],'g.',label='Corrected')
plt.xlabel('Exposure number'),plt.ylabel('base_SdssShape_xx')

plt.subplot(133)
plt.plot(bf_dat[:,nfoo,3],'r.',label='Uncorrected')
plt.plot(bf_dat[:,nfoo,5],'g.',label='Corrected')
plt.xlabel('Exposure number'),plt.ylabel('base_SdssShape_yy')

In [ ]:
# +TODO add re-scaled stamp subtraction comparison
# +TODO other ways of doing matching, catalog stacking
# should this analysis focus on only one stamp? realism in wide-field correction, simplicity in stamp by stamp...